In [9]:
from pdf_page_annotator import PDFAnnotator

annotator = PDFAnnotator('doefund.pdf', verbose=True)

Initialized PDFAnnotator with doefund.pdf
Initialized PDFAnnotator with doefund.pdf


In [10]:
annotator.run_extraction_pipeline()

Page 0 ...
Page 0 ...
Page 1 ...
Page 1 ...
Page 2 ...
Page 2 ...
Page 3 ...
Page 3 ...
Page 4 ...
Page 4 ...
Page 5 ...
Page 5 ...
Page 6 ...
Page 6 ...
Finished annotating table of contents
Finished annotating table of contents
Extracting contents ...
Extracting contents ...
Extracting contents from page 2 ...
Extracting contents from page 2 ...
Extracting contents from page 3 ...
Extracting contents from page 3 ...
Extracting contents from page 4 ...
Extracting contents from page 4 ...
Extracting contents from page 5 ...
Extracting contents from page 5 ...
Extracted contents
Extracted contents
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting 

In [11]:
content = annotator.contents
content[0].unique_title, content[0].start_page, content[0].end_page

('I. Funding Opportunity Description', 1, 12)

In [12]:
# extract all the tuples of (start_page, end_page) for each unique_title
heading_tuples = []
for c in content:
    heading_tuples.append((c.unique_title, c.start_page, c.end_page))
len(heading_tuples)

118

In [13]:
# export the tuples into a csv file names toc.csv
import csv
with open('toc.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(heading_tuples)

In [14]:
PROMPT = """Given are the Table of Contents of an RFP document. Based on the user query, return the section and it's contents where the requested information is found.
Each table of content tuple contains the unique heading, start page and end page of the section.
Table of Contents:
{toc}
Query:
{query}
---
Return a list of tuples as expressed above.
Results:"""
print(PROMPT.format(toc=heading_tuples, query="Where can I find all the evaluation criteria?"))

Given are the Table of Contents of an RFP document. Based on the user query, return the section and it's contents where the requested information is found.
Each table of content tuple contains the unique heading, start page and end page of the section.
Table of Contents:
[('I. Funding Opportunity Description', 1, 12), ('A. Background and Context', 1, 2), ('i. Program Purpose', 2, 2), ('ii. Technology Space and Strategic Goals', 5, 5), ('B. Topic Areas', 6, 6), ('C. Applications Specifically Not of Interest', 11, 11), ('D. R&D Community Benefits Plan (CBP)', 12, 12), ('E. Authorizing Statutes', 12, 12), ('F. Notice of Bipartisan Infrastructure Law– Specific Requirements', 13, 13), ('II. Award Information', 14, 15), ('A. Award Overview', 14, 14), ('B. DOE Funding Agreements', 14, 15), ('III. Eligibility Information', 15, 15), ('A. Eligible Applicants', 15, 16), ('B. Cost Sharing', 16, 19), ('C. Compliance Criteria', 20, 20), ('D. Responsiveness Criteria', 21, 21), ('E. Other Eligibility 

In [15]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")
# response = llm.complete(PROMPT.format(toc=heading_tuples, query="Where can I find all the evaluation criteria? Return the section start and end item.")).text

In [19]:
response

"[('V. Application Review Information', 56, 61), ('A. Technical Review Criteria', 56, 61), ('i. Concept Papers', 56, 61), ('ii. Full Applications', 57, 61)]"

In [20]:
import ast
response = ast.literal_eval(response)
response

[('V. Application Review Information', 56, 61),
 ('A. Technical Review Criteria', 56, 61),
 ('i. Concept Papers', 56, 61),
 ('ii. Full Applications', 57, 61)]

In [10]:
page_start = 27
page_end = 34
page_start, page_end

(27, 34)

In [12]:
from pypdf import PdfReader
reader = PdfReader('egsearch.pdf')
text = reader.pages[page_start:page_end]
text = [t.extract_text() for t in text]
text

['SOLICITATION NUMBER 75N992-24-Q -00001  \nPage 28 of 42  \nSECTION 4 – EVALUATIO N FACTORS AND AWARD PROCESS  \n4.1 Evaluation  Criteria  \n \nThis procurement is being conducted under FAR Subpart  13.5 . FAR Part 15, and specifically \nsource selection procedures at FAR  Subpart  15.3 , do not apply  to evaluation of quotations and \nwill not be used.  \n \nThe Government will award  the contract  to the quoter  that represents the overall best value to \nmeet the Government’s needs. Best Value is defined as the expected outcome of an acquisition that, in the Government’s estimation, provides the greatest overall benefit in response to the requirement, in accordance with FAR 2.101. The Government intends to determine the best value outcome by utilizing a Tradeoff Process to consider making award to other than the lowest \nprice or other than the highest technically r ated quoter . \n THE GOVERNMENT INTENDS TO ISSUE AWARD WITHOUT ENGAGING IN \nEXCHANGES  WITH VENDORS, BUT RESERVES TH

In [18]:
EXTRACTION_PROMPT = """For the given excerpt from an RFP document, it is suspected that it contains the evaluation criteria and hard requirements that the application will be evaluated against.
As an advanced language model, your job is to remove the fluff and return these criteria and requirements.
Output a list of these criteria and requirements with each item separated by a newline.
Ensure that the items are hard conditionals and not soft language.
Excerpt:
{excerpt}
---
Start each line with a hyphen and a space. Start each item with something like, "The applicant must" or "The application should" or something similar.
Results:"""
__EXTRACTION_PROMPT = """For the given excerpt from an RFP document, it is suspected that it contains the evaluation criteria and hard requirements that the application will be evaluated against.
As an advanced language model, your job is to remove the fluff and return these criteria and requirements.
Output a list of these criteria and requirements with each item separated by a newline.
Ensure that the items are hard conditionals and not soft language.
Excerpt:
{excerpt}
---
Start each line with a hyphen and a space. Each item should have a cause and effect format like, "The application will be evaluated on ... so the applicant must ..." or "The application should ... so the applicant must ..."
Results:"""
___EXTRACTION_PROMPT = """For the given excerpt from an RFP document, it is suspected that it contains the evaluation criteria and hard requirements that the application will be evaluated against.
As an advanced language model, your job is to remove the fluff and return these criteria and requirements.
Output a list of these criteria and requirements with each item separated by a newline.
Ensure that the items are hard conditionals and not soft language.
Excerpt:
{excerpt}
---
Start each line with a hyphen and a space. Each item should have a cause and effect format like, "The application will be evaluated on ... so the applicant must ..." or, if necessary, "The application will not be considered if ... so the applicant must ..."
Results:"""
print(EXTRACTION_PROMPT.format(excerpt=text[0]))

For the given excerpt from an RFP document, it is suspected that it contains the evaluation criteria and hard requirements that the application will be evaluated against.
As an advanced language model, your job is to remove the fluff and return these criteria and requirements.
Output a list of these criteria and requirements with each item separated by a newline.
Ensure that the items are hard conditionals and not soft language.
Excerpt:
SOLICITATION NUMBER 75N992-24-Q -00001  
Page 28 of 42  
SECTION 4 – EVALUATIO N FACTORS AND AWARD PROCESS  
4.1 Evaluation  Criteria  
 
This procurement is being conducted under FAR Subpart  13.5 . FAR Part 15, and specifically 
source selection procedures at FAR  Subpart  15.3 , do not apply  to evaluation of quotations and 
will not be used.  
 
The Government will award  the contract  to the quoter  that represents the overall best value to 
meet the Government’s needs. Best Value is defined as the expected outcome of an acquisition that, in the G

In [19]:
llm.model

'gpt-3.5-turbo'

In [20]:
_conditions = []
for t in text:
    response = llm.complete(EXTRACTION_PROMPT.format(excerpt=t)).text
    _conditions.append(response)

In [21]:
# export the conditions into a txt file
with open('eval_conditionss.txt', 'w') as file:
    file.write('\n'.join(_conditions))